In [1]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
# import flask
# import pandas_datareader.data as web

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [3]:
?yf.download

Signature:
yf.download(
    tickers,
    start=None,
    end=None,
    actions=False,
    threads=True,
    group_by='column',
    auto_adjust=False,
    back_adjust=False,
    progress=True,
    period='max',
    interval='1d',
    prepost=False,
    proxy=None,
    rounding=False,
    **kwargs,
)
Docstring:
Download yahoo tickers
:Parameters:
    tickers : str, list
        List of tickers to download
    period : str
        Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        Either Use period parameter or use start and end
    interval : str
        Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        Intraday data cannot extend last 60 days
    start: str
        Download start date string (YYYY-MM-DD) or _datetime.
        Default is 1900-01-01
    end: str
        Download end date string (YYYY-MM-DD) or _datetime.
        Default is now
    group_by : str
        Group by 'ticker' or 'column' (default)
    prepost : bool
        Include Pre and Post 

In [4]:
interval='5m'
period='5d'
cd = 'CL=F'

In [5]:
# fig = go.Figure(data=[go.Candlestick(x=df.index,
#                 open=df[('Open', cd)],
#                 high=df[('High',cd)],
#                 low=df[('Low',cd)],
#                 close=df[('Close',cd)])])
# fig.update_layout(xaxis_rangeslider_visible=False, title=f"{cd} {interval} Intervals")
# fig.show()

In [6]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        id='toggle-rangeslider',
        options=[{'label': 'Include Rangeslider', 
                  'value': 'slider'}],
        value=[None]
    ),
    dcc.Dropdown(
        id='dd_code',
        options=[
            {'label': 'CL<0>','value': 'CL=F'},
            {'label': 'RB<0>','value': 'RB=F'},
            {'label': 'HO<0>','value': 'HO=F'}
        ],
        value='CL=F'
    ),
    dcc.Dropdown(
        id='dd_int',
        options=[
            {'label': '1m','value': '1m'},
            {'label': '5m','value': '5m'},
            {'label': '15m','value': '15m'}
        ],
        value='5m'
    ),
    dcc.Dropdown(
        id='dd_period',
        options=[
            {'label': '1d','value': '1d'},
            {'label': '5d','value': '5d'},
            {'label': '1mo','value': '1mo'}
        ],
        value='5d'
    ),
    dcc.Graph(id="graph"),
])

@app.callback(
    Output("graph", "figure"), 
    [Input("toggle-rangeslider", "value"), 
     Input("dd_code", "value"),
     Input("dd_int", "value"),
     Input("dd_period", "value")
    ])
def display_candlestick(value, cd, interval, period):
    df = yf.download(cd, period=period, interval=interval)
    
    if (df.shape[0] == 0) & (period=='1d'):
        fig = go.Figure()
        fig.update_layout(title="Markets are closed for today, please choose 5d or greater for your period")
    else:
        fig = go.Figure(data=[go.Candlestick(x=df.index,
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'])])
        # fig.update_layout(xaxis_rangeslider_visible=False)

        fig.update_layout(
            xaxis_rangeslider_visible='slider' in value,
            title=f"{cd} {interval} Intervals"
        )

    return fig

app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
